In [ ]:
import sys, time, os, asyncio, glob
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import pickle as pkl
from astropy.time import Time, TimeDelta
from lsst_efd_client import EfdClient
from lsst.summit.utils.efdUtils import getEfdData
from lsst.ts.xml.tables.m1m3 import FATable, FAIndex, force_actuator_from_id, actuator_id_to_index

In [ ]:
#from lsst.ts.aos.utils.m1m3 import m1m3_aav

In [ ]:
client = EfdClient("usdf_efd")

In [ ]:
tests = [["2023-12-15T23:24", "2023-12-15T23:40", "40%", "red-red-red-green, e.g. no compensations"], 
         ["2023-12-15T23:49", "2023-12-16T00:05", "40%", "red-green-red-green - HP load cells only, balance forces"], 
         ["2023-12-19T02:32", "2023-12-19T02:45", "40%", "green-red-green-green - DC Accelerometers, TMA velocity"], 
         ["2023-12-20T08:08", "2023-12-20T08:21", "40%", "red-red-red-green, testing PID freeze without compensation, TMA faulted"], 
         ["2023-12-21T23:52", "2023-12-22T00:03", "40%", "green-green-green-green - all compensation"]]

In [ ]:
for [start, end, speed, name] in tests:
    start = Time(start, scale='utc')
    end = Time(end, scale='utc')

    ret = await client.select_time_series(
        "lsst.sal.MTM1M3.hardpointActuatorData",
        ["timestamp"]
        + [f"measuredForce{hp}" for hp in range(6)]
        + [f"f{a}" for a in "xyz"]
        + [f"m{a}" for a in "xyz"],
        start,
        end,
        )

    print(speed, name)
    for hp in range(6):
        max = np.max(ret[f"measuredForce{hp}"])
        min = np.min(ret[f"measuredForce{hp}"])
        print(f"measuredForce{hp} max = {max}")
        print(f"measuredForce{hp} min = {min}")
    print()
    print()
    print(speed, name)
    for a in "xyz":
        max = np.max(ret[f"f{a}"])
        print(f"a{a} max = {max}")
        max = np.max(ret[f"m{a}"])
        print(f"m{a} max = {max}")
        min = np.min(ret[f"f{a}"])
        print(f"a{a} min = {min}")
        min = np.min(ret[f"m{a}"])
        print(f"m{a} min = {min}")
    print()
    print()

In [ ]:
fig, axs = plt.subplots(5,2,figsize=(8, 20))
plt.suptitle("Force actuators maximum following errors", fontsize=24)
plt.subplots_adjust(hspace=0.3, wspace=0.5)
for i, [start, end, speed, name] in enumerate(tests):
    start = Time(start, scale='utc')
    end = Time(end, scale='utc')
    ids = []
    primary_errors = []
    secondary_errors = []
    for index in range(len(FATable)):
        try:
            id = FATable[index].actuator_id
            fa = force_actuator_from_id(id)
            primary_bump = f"primaryTest{fa.index}"
            primary_follow = f"primaryCylinderFollowingError{fa.index}"
            primary_force = f"primaryCylinderForce{fa.index}"
            if fa.actuator_type.name == 'DAA':
                secondary_bump = f"secondaryTest{fa.s_index}"
                secondary_force = f"secondaryCylinderForce{fa.s_index}"
                secondary_follow = f"secondaryCylinderFollowingError{fa.s_index}"
                secondary_name = fa.orientation.name
            else:
                secondary_name = None
                secondary_force = None
                secondary_follow = None
            forces = await client.select_time_series("lsst.sal.MTM1M3.forceActuatorData", \
                                                     [primary_force, primary_follow], start, end)
    
            primary_error = np.max(abs(forces[primary_follow].values))
            ids.append(id)
            primary_errors.append(primary_error)
            if secondary_follow:
                forces = await client.select_time_series("lsst.sal.MTM1M3.forceActuatorData", \
                                                     [secondary_force, secondary_follow], start, end)
                secondary_error = np.max(abs(forces[secondary_follow].values))
                secondary_errors.append(secondary_error)
            else:
                secondary_errors.append(None)
            print(f"ID {id} done")
        except:
            print(f"ID {id} failed")
    axs[i][0].set_title(name, fontsize=10, loc='left')
    axs[i][0].scatter(ids, primary_errors)
    axs[i][0].set_ylim(0,200)
    axs[i][0].set_xlabel("Actuator ID")
    axs[i][0].set_ylabel("Max primary following error (N)")
    #axs[i][1].set_title(name, fontsize=10)
    axs[i][1].scatter(ids, secondary_errors)
    axs[i][1].set_ylim(0,200)
    axs[i][1].set_xlabel("Actuator ID")
    axs[i][1].set_ylabel("Max secondary following error (N)")
plt.savefig("/home/c/cslage/u/MTM1M3/data/Actuator_Following_Errors_22Dec23.pdf")    